In [ ]:
import pandas as pd

In [ ]:
# Specify the file name and optionally the sheet name or index
file_name = '/Users/shawnpana/Documents/GitHub/sushihacks2025/backend/data/41598_2021_97043_MOESM2_ESM.xlsx'
df = pd.read_excel(file_name) # Reads the sheet named 'Sheet1'
# Or to read the first sheet by index:
# df = pd.read_excel(file_name, sheet_name=0)

NameError: name 'pd' is not defined

In [ ]:
# print(df.columns)
# print the number of rows in this dataframe
# print(len(df))

# make the columns lowercase
df.columns = [col.lower() for col in df.columns]
print(df.columns)



Index(['severity', 'mild', 'mod', 'sev', 'age', 'bmi', 'csa', 'pb', 'duration',
       'nrs', 'sex', 'side', 'diabetes', 'np', 'weakness'],
      dtype='object')


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [37]:
def to_numeric_severity(severity):
    if severity == 'mild':
        return 0
    elif severity == 'moderate':
        return 1
    elif severity == 'severe':
        return 2
    else:
        return -1  # Unknown severity
    


In [ ]:
df['severity_numeric'] = df['severity'].apply(to_numeric_severity)

# Keep only the required features plus severity_numeric
# Note: weakness will be replaced with grip_strength and pinch_strength later
required_features = ['age', 'bmi', 'sex', 'diabetes', 'duration', 'nrs', 'weakness', 'severity_numeric']
df = df[required_features]

In [39]:
df.head()

,age,bmi,sex,duration,nrs,weakness,severity_numeric
0,65,21.093750,1,3,2,0,0
1,68,20.415225,0,12,6,0,2
2,68,20.415225,0,12,3,0,0
3,63,28.507522,1,1,4,0,0
4,87,22.939751,1,1,2,0,0


In [40]:
df.head()

,age,bmi,sex,duration,nrs,weakness,severity_numeric
0,65,21.093750,1,3,2,0,0
1,68,20.415225,0,12,6,0,2
2,68,20.415225,0,12,3,0,0
3,63,28.507522,1,1,4,0,0
4,87,22.939751,1,1,2,0,0


In [41]:
# Check weakness distribution
print("Weakness distribution:")
print(df['weakness'].value_counts())
print(f"Weakness rate: {df['weakness'].mean():.2%}")

print("\nDataset info:")
print(df.dtypes)

Weakness distribution:
weakness
0    843
1    194
Name: count, dtype: int64
Weakness rate: 18.71%

Dataset info:
age                   int64
bmi                 float64
sex                   int64
duration              int64
nrs                   int64
weakness              int64
severity_numeric      int64
dtype: object


In [42]:
import numpy as np

def create_strength_values(age, sex, weakness_status):
    """
    Create realistic grip and pinch strength values based on clinical norms
    and weakness status.
    
    Args:
        age: patient age
        sex: 0 for female, 1 for male
        weakness_status: 0 for no weakness, 1 for weakness
    
    Returns:
        tuple: (grip_strength, pinch_strength) in kg
    """
    # Calculate expected values based on clinical norms
    if sex == 1:  # Male
        expected_grip = 46.0 - (0.24 * age)
        expected_pinch = 9.0 - (0.05 * age)
    else:  # Female
        expected_grip = 27.0 - (0.18 * age)
        expected_pinch = 6.0 - (0.04 * age)
    
    # Add some natural variation (±10% standard deviation)
    grip_variation = np.random.normal(0, 0.1)
    pinch_variation = np.random.normal(0, 0.1)
    
    if weakness_status == 1:
        # If weakness present, create deficit ≥20% (typically 20-40%)
        grip_deficit = np.random.uniform(0.20, 0.40)
        pinch_deficit = np.random.uniform(0.20, 0.35)
        
        grip_strength = expected_grip * (1 - grip_deficit) * (1 + grip_variation)
        pinch_strength = expected_pinch * (1 - pinch_deficit) * (1 + pinch_variation)
    else:
        # If no weakness, create deficit <20% (typically 0-15%)
        grip_deficit = np.random.uniform(0.0, 0.15)
        pinch_deficit = np.random.uniform(0.0, 0.15)
        
        grip_strength = expected_grip * (1 - grip_deficit) * (1 + grip_variation)
        pinch_strength = expected_pinch * (1 - pinch_deficit) * (1 + pinch_variation)
    
    # Ensure positive values
    grip_strength = max(grip_strength, 1.0)
    pinch_strength = max(pinch_strength, 0.5)
    
    return round(grip_strength, 1), round(pinch_strength, 1)

# Set random seed for reproducibility
np.random.seed(42)

# Create strength columns
strength_data = []
for idx, row in df.iterrows():
    grip, pinch = create_strength_values(row['age'], row['sex'], row['weakness'])
    strength_data.append({'grip_strength': grip, 'pinch_strength': pinch})

strength_df = pd.DataFrame(strength_data)
df['grip_strength'] = strength_df['grip_strength']
df['pinch_strength'] = strength_df['pinch_strength']

print("Sample of new strength data:")
print(df[['age', 'sex', 'weakness', 'grip_strength', 'pinch_strength']].head(10))


Sample of new strength data:
   age  sex  weakness  grip_strength  pinch_strength
0   65    1         0           28.4             5.2
1   68    0         0           14.3             2.8
2   68    0         0           17.0             3.0
3   63    1         0           28.6             6.0
4   87    1         0           24.1             3.6
5   87    1         0           21.6             4.5
6   55    1         0           36.5             5.6
7   64    1         0           26.3             5.7
8   64    1         0           25.8             5.3
9   60    1         0           31.4             4.6


In [ ]:
# Update feature selection to use strength measures instead of weakness
# Remove weakness column and keep the new strength columns
final_features = ['age', 'bmi', 'sex', 'diabetes', 'duration', 'nrs', 'grip_strength', 'pinch_strength', 'severity_numeric']
df = df[final_features]

print("Updated dataset shape:", df.shape)
print("Final features:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())


Updated dataset shape: (1037, 8)
Final features: ['age', 'bmi', 'sex', 'duration', 'nrs', 'grip_strength', 'pinch_strength', 'severity_numeric']

First few rows:
   age        bmi  sex  duration  nrs  grip_strength  pinch_strength  \
0   65  21.093750    1         3    2           28.4             5.2   
1   68  20.415225    0        12    6           14.3             2.8   
2   68  20.415225    0        12    3           17.0             3.0   
3   63  28.507522    1         1    4           28.6             6.0   
4   87  22.939751    1         1    2           24.1             3.6   

   severity_numeric  
0                 0  
1                 2  
2                 0  
3                 0  
4                 0  


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

target_col = "severity_numeric"

X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=2/3, stratify=y_temp, random_state=42
)

model = XGBClassifier(
    objective="multi:softprob",  # probabilities for each class
    num_class=3,
    eval_metric=["mlogloss", "merror"],
    n_estimators=3000,           # large cap + early stopping will pick the best
    learning_rate=0.05,          # smaller LR + early stopping is robust
    max_depth=6,                 # tuneable
    min_child_weight=2,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    tree_method="hist",          # fast
    random_state=42,
    n_jobs=-1,
)

model.fit(
    X_train, y_train,
    # sample_weight=sample_weight,
    eval_set=[(X_valid, y_valid)],
    verbose=False,
    # early_stopping_rounds=100
)

y_pred = model.predict(X_test)
print("\nClassification report (test):")
print(classification_report(y_test, y_pred, digits=4))

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)


Out-of-Bag Score: 0.7007
Number of trees: 500

Feature Importance:
  nrs: 0.2346
  duration: 0.2044
  grip_strength: 0.1472
  bmi: 0.1471
  pinch_strength: 0.1218
  age: 0.1185
  sex: 0.0263

Classification report (test):
              precision    recall  f1-score   support

           0     0.7568    0.8235    0.7887       102
           1     0.5000    0.3818    0.4330        55
           2     0.7273    0.7843    0.7547        51

    accuracy                         0.6971       208
   macro avg     0.6613    0.6632    0.6588       208
weighted avg     0.6816    0.6971    0.6863       208

Confusion matrix:
 [[84 15  3]
 [22 21 12]
 [ 5  6 40]]


In [ ]:
# Create test data with realistic strength values for a 65-year-old female with some weakness
# Expected grip for 65-year-old female: 27.0 - (0.18 × 65) = 15.3 kg
# Expected pinch for 65-year-old female: 6.0 - (0.04 × 65) = 3.4 kg
# Using values that suggest mild weakness (15-20% deficit)

new_data = pd.DataFrame([{
    "age": 65,
    "bmi": 23.530366,
    "sex": 0,  # female
    "diabetes": 1,
    "duration": 6,
    "nrs": 7,
    "grip_strength": 13.0,  # ~15% below expected (15.3 kg)
    "pinch_strength": 2.9,  # ~15% below expected (3.4 kg)
}])

pred_class = model.predict(new_data)
pred_proba = model.predict_proba(new_data)

print("Predicted class:", pred_class[0])
print("Class probabilities:")
print(f"  Mild (0): {pred_proba[0][0]:.3f}")
print(f"  Moderate (1): {pred_proba[0][1]:.3f}")
print(f"  Severe (2): {pred_proba[0][2]:.3f}")


Predicted class: 1
Class probabilities:
  Mild (0): 0.076
  Moderate (1): 0.668
  Severe (2): 0.256


In [46]:
# Validate that our strength values follow the clinical rules
def validate_weakness_prediction(age, sex, grip_strength, pinch_strength):
    """Validate weakness prediction based on clinical rules"""
    # Calculate expected values
    if sex == 1:  # Male
        expected_grip = 46.0 - (0.24 * age)
        expected_pinch = 9.0 - (0.05 * age)
    else:  # Female
        expected_grip = 27.0 - (0.18 * age)
        expected_pinch = 6.0 - (0.04 * age)
    
    # Calculate deficits
    grip_deficit = (expected_grip - grip_strength) / expected_grip * 100
    pinch_deficit = (expected_pinch - pinch_strength) / expected_pinch * 100
    
    # Apply 20% rule
    weakness_by_grip = grip_deficit >= 20
    weakness_by_pinch = pinch_deficit >= 20
    
    return {
        'expected_grip': expected_grip,
        'expected_pinch': expected_pinch,
        'grip_deficit_pct': grip_deficit,
        'pinch_deficit_pct': pinch_deficit,
        'weakness_by_grip': weakness_by_grip,
        'weakness_by_pinch': weakness_by_pinch,
        'predicted_weakness': weakness_by_grip or weakness_by_pinch
    }

# Test with a few examples from our dataset
print("Validation of strength-based weakness prediction:")
print("="*60)
sample_rows = df.sample(5, random_state=42)
for idx, row in sample_rows.iterrows():
    result = validate_weakness_prediction(
        row['age'], row['sex'], row['grip_strength'], row['pinch_strength']
    )
    print(f"Patient: Age {row['age']}, {'Male' if row['sex']==1 else 'Female'}")
    print(f"  Grip: {row['grip_strength']:.1f} kg (expected: {result['expected_grip']:.1f}, deficit: {result['grip_deficit_pct']:.1f}%)")
    print(f"  Pinch: {row['pinch_strength']:.1f} kg (expected: {result['expected_pinch']:.1f}, deficit: {result['pinch_deficit_pct']:.1f}%)")
    print(f"  Predicted weakness: {result['predicted_weakness']}")
    print()


Validation of strength-based weakness prediction:
Patient: Age 54.0, Male
  Grip: 28.7 kg (expected: 33.0, deficit: 13.1%)
  Pinch: 5.7 kg (expected: 6.3, deficit: 9.5%)
  Predicted weakness: False

Patient: Age 72.0, Female
  Grip: 12.7 kg (expected: 14.0, deficit: 9.5%)
  Pinch: 2.9 kg (expected: 3.1, deficit: 7.1%)
  Predicted weakness: False

Patient: Age 65.0, Female
  Grip: 10.6 kg (expected: 15.3, deficit: 30.7%)
  Pinch: 2.4 kg (expected: 3.4, deficit: 29.4%)
  Predicted weakness: True

Patient: Age 53.0, Female
  Grip: 17.0 kg (expected: 17.5, deficit: 2.6%)
  Pinch: 3.9 kg (expected: 3.9, deficit: -0.5%)
  Predicted weakness: False

Patient: Age 57.0, Male
  Grip: 23.5 kg (expected: 32.3, deficit: 27.3%)
  Pinch: 4.0 kg (expected: 6.2, deficit: 35.0%)
  Predicted weakness: True



In [47]:
df[df['severity_numeric'] == 2]

,age,bmi,sex,duration,nrs,grip_strength,pinch_strength,severity_numeric
1,68,20.415225,0,12,6,14.3,2.8,2
7,64,24.919900,1,3,4,26.3,5.7,2
8,64,24.919900,1,3,4,25.8,5.3,2
26,58,25.558846,1,5,7,22.4,4.9,2
38,76,26.395803,0,2,7,9.5,2.0,2
...,...,...,...,...,...,...,...,...
1021,56,23.530366,1,6,7,34.4,5.4,2
1023,37,27.041644,0,12,9,12.9,3.2,2
1027,64,26.666667,1,12,5,20.8,3.8,2
1029,71,29.242109,0,24,6,9.5,1.8,2


In [ ]:
import pickle

with open("new_model_2.pkl", "wb") as f:
    pickle.dump(model, f)


Model saved as:
- carpal_tunnel_rf_model.pkl (pickle format)
- carpal_tunnel_rf_model.joblib (joblib format)
Features: ['age', 'bmi', 'sex', 'duration', 'nrs', 'grip_strength', 'pinch_strength']
Classes: ['mild', 'moderate', 'severe']
Ready for API integration!
